import sys  
reload(sys)  
sys.setdefaultencoding('ascii')

In [31]:
import pandas as pd
import glob
import timeit

In [3]:
target_df = pd.read_csv('../data/final/HuizhouTitleClassifier-3.csv')

In [4]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23127 entries, 0 to 23126
Data columns (total 3 columns):
url            23127 non-null object
token title    23127 non-null object
flag           23127 non-null int64
dtypes: int64(1), object(2)
memory usage: 722.7+ KB


In [35]:
solr_segment_title_df = pd.read_csv('../data/segments/titles/bosonNLP/solor-bosonNLP-segment-0-21280.csv')

In [38]:
solr_segment_title_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19788 entries, 0 to 19835
Data columns (total 5 columns):
url        19788 non-null object
domain     19788 non-null object
title      19788 non-null object
segment    19788 non-null object
target     0 non-null float64
dtypes: float64(1), object(4)
memory usage: 927.6+ KB


In [37]:
df_no_dup = solr_segment_title_df.drop_duplicates(['segment'])
solr_segment_title_df = df_no_dup.dropna(subset =['segment'])

In [40]:
solr_segment_title_df.to_csv('../data/segments/titles/bosonNLP/solor-bosonNLP-segment-0-21280-1.csv', encoding='utf-8', index=False)
solr_segment_title_df = pd.read_csv('../data/segments/titles/bosonNLP/solor-bosonNLP-segment-0-21280-1.csv')

In [18]:
merged = target_df.merge(target_df, indicator=True, how='outer')

In [22]:
merged_both = merged[merged['_merge'] == 'both']
merged_rightonly = merged[merged['_merge'] == 'right_only']
merged_leftonly = merged[merged['_merge'] == 'left_only']


In [24]:
merged_both.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23127 entries, 0 to 23126
Data columns (total 4 columns):
url            23127 non-null object
token title    23127 non-null object
flag           23127 non-null int64
_merge         23127 non-null category
dtypes: category(1), int64(1), object(2)
memory usage: 745.3+ KB


In [27]:
def get_columns2():
    return ('url', 'domain', 'title', 'segment', 'target', 'source') 
def get_empty_columns2():
    return ('', '', '', '', '', '') 

In [45]:
assigned_df = pd.DataFrame(columns = get_columns2())
idx = 0

In [46]:
unassigned_df = pd.DataFrame(columns = get_columns2())
idx2 = 0

In [47]:
start_time = timeit.default_timer()

for i in range(0, len(solr_segment_title_df)):
    url = solr_segment_title_df.loc[i]['url']
    df_0 = target_df[target_df['url'] == url]
    if len(df_0.values) != 0:
        assigned_df.loc[idx] = get_empty_columns2()
        assigned_df.loc[idx]['url'] = url
        assigned_df.loc[idx]['segment'] = solr_segment_title_df.loc[i]['segment']
        assigned_df.loc[idx]['target'] = df_0.iloc[0]['flag']
        assigned_df.loc[idx]['title'] = solr_segment_title_df.loc[i]['title']
        assigned_df.loc[idx]['source'] = 'solr'
        idx += 1
    else:
        #unsigned target
        unassigned_df.loc[idx2] = get_empty_columns2()
        unassigned_df.loc[idx2]['url'] = url
        unassigned_df.loc[idx2]['segment'] = solr_segment_title_df.loc[i]['segment']
        unassigned_df.loc[idx2]['title'] = solr_segment_title_df.loc[i]['title']
        unassigned_df.loc[idx2]['source'] = 'solr'
        idx2 += 1
        #print 'missed -----', url

elapsed = timeit.default_timer() - start_time

In [48]:
assigned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17282 entries, 0 to 17281
Data columns (total 6 columns):
url        17282 non-null object
domain     17282 non-null object
title      17282 non-null object
segment    17282 non-null object
target     17282 non-null object
source     17282 non-null object
dtypes: object(6)
memory usage: 945.1+ KB


In [49]:
unassigned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2506 entries, 0 to 2505
Data columns (total 6 columns):
url        2506 non-null object
domain     2506 non-null object
title      2506 non-null object
segment    2506 non-null object
target     2506 non-null object
source     2506 non-null object
dtypes: object(6)
memory usage: 137.0+ KB


In [50]:
assigned_df.to_csv('assigned_target.csv', encoding='utf-8', index=False)
unassigned_df.to_csv('unssigned_target.csv', encoding='utf-8', index=False)

In [62]:
assigned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22933 entries, 0 to 22932
Data columns (total 5 columns):
url        22933 non-null object
title      22933 non-null object
segment    22933 non-null object
target     22933 non-null object
source     17812 non-null object
dtypes: object(5)
memory usage: 1.0+ MB


In [63]:
unassigned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 193
Data columns (total 5 columns):
url        194 non-null object
title      194 non-null object
segment    194 non-null object
target     194 non-null object
source     194 non-null object
dtypes: object(5)
memory usage: 9.1+ KB


In [73]:
new_df3 = new_df2.append(new_df)

In [74]:
new_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23127 entries, 0 to 22932
Data columns (total 5 columns):
url        23127 non-null object
title      23127 non-null object
segment    23127 non-null object
target     23127 non-null object
source     18006 non-null object
dtypes: object(5)
memory usage: 1.1+ MB


In [77]:
#df_no_dup = new_df3.dropna(subset =['url']) 
#df_no_dup = df_no_dup.dropna(subset =['segment']) 
df_no_dup = new_df3.drop_duplicates(['url'])
df_no_dup = df_no_dup.drop_duplicates(['segment'])

In [78]:
df_no_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23116 entries, 0 to 22932
Data columns (total 5 columns):
url        23116 non-null object
title      23116 non-null object
segment    23116 non-null object
target     23116 non-null object
source     18006 non-null object
dtypes: object(5)
memory usage: 1.1+ MB


In [79]:
df_no_dup.to_csv('../data/final/HuizhouTitleSegment.csv', encoding='utf-8', index=False)

In [155]:
new_df.head(100)

,url,title,segment,target,source
0,http://cul.anhuinews.com/system/2009/05/05/002...,我是安徽徽州人――胡适的一生-胡适-中安在线-徽文化,我 是 安徽 徽州 人 ―― 胡 适 的 一生 - 胡 适 -\n,1,solr
1,http://cul.anhuinews.com/system/2009/09/07/002...,用镜头纪录最美的黄山 徽文化频道 中安在线,用 镜头 纪录 最 美 的 黄山\n,0,solr
2,http://cul.anhuinews.com/system/2010/03/01/002...,又到义写春联时-书法-中安在线-徽文化,又 到 义 写 春联 时 - 书法 -\n,0,solr
3,http://cul.anhuinews.com/system/2010/04/12/002...,29、办事成本高乎哉？-老M神侃-中安在线-徽文化,29 、 办事 成本 高 乎 哉 ？ - 老M 神侃 -\n,0,solr
4,http://cul.anhuinews.com/system/2010/04/12/002...,7、不要以为“干好了就不是人”-老M神侃-中安在线-徽文化,7 、 不要 以为 “ 干 好 了 就 不 是 人 ” - 老M 神侃 -\n,0,solr
5,http://cul.anhuinews.com/system/2011/01/22/003...,《牛郎织女》精彩剧照--中安在线-徽文化,《 牛郎织女 》 精彩 剧照 --\n,0,solr
6,http://cul.anhuinews.com/system/2011/05/04/004...,美丽-苏北-中安在线-徽文化,美丽 - 苏北 -\n,0,solr
7,http://cul.anhuinews.com/system/2013/11/19/006...,走进博物馆 留住最美的秋天童话记忆-博物馆-中安在线-徽文化,走 进 博物馆 留住 最 美 的 秋天 童话 记忆 - 博物馆 -\n,0,solr
8,http://www.ahage.net/html/shiping/31841_3.html,台儿庄大捷壮烈：无墙不饮弹 无土不沃血(3)_安徽文化网 | 安徽文化人的网上家园，网上最大...,台儿庄 大捷 壮烈 ： 无 墙 不 饮弹 无土 不 沃 血 ( 3 )\n,0,solr
9,http://www.ahage.net/huidiao/27103.html,彭劲杰与木雕的二十年“不了情”_我的网站,彭 劲杰 与 木雕 的 二十 年 “ 不 了 情 ”\n,1,solr


In [156]:
new_df2.head(100)

,url,title,segment,target,source
0,http://cul.anhuinews.com/system/2014/06/30/006...,漫漫古皖道\n,漫漫 古 皖道\n,0,general
1,http://www.ahage.net/huishang/5306.html,屯溪商埠与徽州木商\n,屯溪 商埠 与 徽州 木商\n,1,general
2,http://www.ahage.net/huiju/586.html,徽剧同京剧是什么关系\n,徽剧 同 京剧 是 什么 关系\n,1,general
3,http://www.ahage.net/huiju/353.html,徽班──中国戏剧史上的奇观\n,徽班 ── 中国 戏剧史 上 的 奇观\n,1,general
4,http://www.ahage.net/huixue/226.html,徽州学·敦煌学·区域史\n,徽州 学 · 敦煌学 · 区域 史\n,1,general
5,http://www.ahage.net/chawenhua/19363.html,郑板桥松萝结茶缘\n,郑 板桥 松萝 结 茶缘\n,0,general
6,http://www.ahage.net/chawenhua/28896.html,细品屯绿茗香\n,细品 屯 绿 茗香\n,1,general
7,http://cul.anhuinews.com/system/2014/06/11/006...,休宁松萝与哥德堡号\n,休宁 松萝 与 哥德堡号\n,1,general
8,http://www.ahage.net/huixue/200.html,徽学漫议(什么是徽学)\n,徽学 漫 议 ( 什么是 徽学 )\n,1,general
9,http://cul.anhuinews.com/system/2009/02/11/002...,舒绣文最后的日子\n,舒 绣 文 最后 的 日子\n,1,general


In [19]:
print idx

0


In [18]:
df_no_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23091 entries, 0 to 23090
Data columns (total 5 columns):
segment    23091 non-null object
source     17834 non-null object
target     23091 non-null int64
title      23091 non-null object
url        23091 non-null object
dtypes: int64(1), object(4)
memory usage: 1.1+ MB


In [20]:

for i in range(0, len(df_no_dup)):
    url = df_no_dup.loc[i]['url']
    df_0 = final_df[final_df['url'] == url]
    if len(df_0.values) == 0:
        #print 'missed -----', url
        new_df.loc[idx] = get_empty_columns2()
        new_df.loc[idx]['url'] = url
        new_df.loc[idx]['segment'] = df_no_dup.loc[i]['segment']
        new_df.loc[idx]['target'] = df_no_dup.loc[i]['target']
        new_df.loc[idx]['title'] = df_no_dup.loc[i]['title']
        new_df.loc[idx]['source'] = df_no_dup.loc[i]['source']
        new_df.loc[idx]['flag'] = df_no_dup.loc[i]['target']
        idx += 1

In [38]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3601 entries, 0 to 3600
Data columns (total 5 columns):
url        3601 non-null object
title      3601 non-null object
segment    3601 non-null object
target     3601 non-null object
source     3417 non-null object
dtypes: object(5)
memory usage: 168.8+ KB


In [22]:
print idx

3601


In [23]:
new_df.head(100)

,url,title,segment,target,source
0,http://www.360doc.com/content/10/1229/16/11473...,让游客畅游“徽州文化之海”,让 游客 畅游 “ 徽州 文化 之 海 ”\n,1,cse
1,http://www.360doc.com/content/15/1107/22/87804...,徽商和徽州文化的是如何起源的？,徽商 和 徽州 文化 的 是 如何 起源 的 ？\n,1,cse
2,http://www.360doc.com/content/13/1130/15/13701...,徽州文化的现存及其原因和价值,徽州 文化 的 现存 及其 原因 和 价值\n,1,cse
3,http://www.360doc.com/content/16/0325/09/35452...,徽州的居所,徽州 的 居所\n,1,cse
4,http://www.360doc.com/content/16/1015/08/28672...,秋冬季节游黄山必去的地方,秋 冬 季节 游 黄山 必 去 的 地方\n,1,cse
5,http://www.hzwh.com/article.asp?id=1332,郑寒:刀尖舞动徽文化,郑 寒 : 刀尖 舞动 徽 文化\n,1,cse
6,http://www.hzwh.com/toplistb.asp?id=234,徽派版画,徽派 版画\n,1,cse
7,http://www.hzwh.com/toplistb.asp?id=245,新安医学,新安 医学\n,1,cse
8,http://www.hzwh.com/toplistb.asp?id=248,徽州竹编,徽州 竹编\n,1,cse
9,http://www.hzwh.com/toplistb.asp?id=231,徽派建筑,徽派 建筑\n,1,cse


In [74]:
new_df['target'].sum()

561

In [24]:
new_df.to_csv('test100.csv', encoding='utf-8', index=False)

In [28]:
df = pd.read_csv('../data/segments/final/solr-huizhou-more-all.csv')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 191
Data columns (total 5 columns):
url        192 non-null object
segment    192 non-null object
title      192 non-null object
target     192 non-null int64
source     192 non-null object
dtypes: int64(1), object(4)
memory usage: 9.0+ KB


In [39]:
idx = 0
cnt = 0
for i in range(0, len(df)):
    url0 = df.loc[i]['url']
    url = url0[:-5]
    print url
    if url in new_df['url']:
        cnt += 1
    else: 
        idx += 1
    

http://www.ahage.net/news/29365
http://www.ahage.net/fengsu/22238
http://www.ahage.net/fengsu/22359
http://www.ahage.net/lunwen/21999
 http://www.ahage.net/lunwen/4130
http://www.ahage.net/huangshan/331
http://www.ahage.net/huixue/6465
http://www.ahage.net/huixue/huizhoulangqiao
http://www.ahage.net/huixue/5133
http://www.ahage.net/huishang/luelunhuishangdebiehaoyuzhaiming
http://www.ahage.net/huishang/zhoumeihong-zhiyanyiren-xixianlaohukaiwenmochangchangchang
http://www.ahage.net/lunwen/5890
http://www.ahage.net/huicai/1999
http://www.ahage.net/huicai/2006
http://www.ahage.net/huidiao/1273
http://www.ahage.net/huishang/5015
http://www.ahage.net/huixue/2235
http://www.ahage.net/huixue/5071
http://www.ahage.net/lunwen/14579
http://www.ahage.net/lunwen/22209
http://www.ahage.net/lunwen/5384
http://www.ahage.net/mingren/1720
http://www.ahage.net/mingren/6926
http://www.ahage.net/shuhua/14815
http://www.ahage.net/lvyou/21991
http://www.ahage.net/mingren/1082
http://www.ahage.net/shuhua/106

In [37]:
print cnt, idx

0 192


In [33]:
df = pd.read_csv('../data/segments/solr-huizhou-more-all-bak.csv')

In [34]:
df.head()

,url,token title,flag
0,http://www.ahage.net/news/29365.html,旌德 漆画 入选 安徽省 第四 批 非 遗 保护 名录,1
1,http://www.ahage.net/fengsu/22238.html,民间 婚俗 见证 徽州 微型 “ 撒 帐 书 ” ( 附 撒 帐 歌 ),1
2,http://www.ahage.net/fengsu/22359.html,上世纪 五六十年代 歙北 农村 春节 文化 活动 及其 影响,1
3,http://www.ahage.net/lunwen/21999.html,徽州 人 的 和谐 思维 及 实践,1
4,http://www.ahage.net/lunwen/4130.html,儒 、 商 互济 的 精神 家园 —— 《 八 说 昌 溪 》,1


In [35]:
df['segment'] = df['token title']
df['target'] = df['flag']
df['title'] = df['token title'].str.replace(' ', '')
df['source'] = 'solr'

In [37]:
df_0 = df[['url', 'segment', 'title', 'target', 'source']]

In [38]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 191
Data columns (total 5 columns):
url        192 non-null object
segment    192 non-null object
title      192 non-null object
target     192 non-null int64
source     192 non-null object
dtypes: int64(1), object(4)
memory usage: 9.0+ KB


In [39]:
df_0.head()

,url,segment,title,target,source
0,http://www.ahage.net/news/29365.html,旌德 漆画 入选 安徽省 第四 批 非 遗 保护 名录,旌德漆画入选安徽省第四批非遗保护名录,1,solr
1,http://www.ahage.net/fengsu/22238.html,民间 婚俗 见证 徽州 微型 “ 撒 帐 书 ” ( 附 撒 帐 歌 ),民间婚俗见证徽州微型“撒帐书”(附撒帐歌),1,solr
2,http://www.ahage.net/fengsu/22359.html,上世纪 五六十年代 歙北 农村 春节 文化 活动 及其 影响,上世纪五六十年代歙北农村春节文化活动及其影响,1,solr
3,http://www.ahage.net/lunwen/21999.html,徽州 人 的 和谐 思维 及 实践,徽州人的和谐思维及实践,1,solr
4,http://www.ahage.net/lunwen/4130.html,儒 、 商 互济 的 精神 家园 —— 《 八 说 昌 溪 》,儒、商互济的精神家园——《八说昌溪》,1,solr


In [40]:
df_0.to_csv('../data/segments/final/solr-huizhou-more-all.csv', encoding='utf-8', index=False)